# LabLivre


## Gather data


In [348]:
# data gathering script was ran separately

import json
import time

import requests


def get_data(page: int) -> dict:
    url = "https://api.obrasgov.gestao.gov.br/obrasgov/api/projeto-investimento"
    params = {"uf": "DF", "pagina": page, "tamanhoDaPagina": 100}
    headers = {"accept": "*/*"}

    response = requests.get(url, params=params, headers=headers)

    if response.status_code != 200:
        raise Exception(f"Failed to get data: {response.status_code}")

    return response.json()


def main():
    # inelegant but expedient. surely they dont have 10000 items
    for page in range(100):
        response = get_data(page)
        print(f"Page {page} processed")

        # save raw data first so if there are issues, we don't depend on the api
        with open(f"data/data-{page}.json", "w", encoding="utf-8") as f:
            json.dump(response["content"], f, indent=4)
        print(f"Data saved to data/data-{page}.json")

        time.sleep(1)

        page += 1


# if __name__ == "__main__":
#     main()

## Data cleaning


In [349]:
import pandas as pd
import glob


def load_json_files() -> list[dict]:
    """Load and combine all JSON files from data directory into a DataFrame"""
    json_files = glob.glob("data/data-*.json")
    all_records = []

    for file in json_files:
        with open(file, "r") as f:
            records = json.load(f)
            all_records.extend(records)

    return all_records


df = pd.DataFrame(load_json_files())

df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 834 entries, 0 to 833
Data columns (total 31 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   idUnico                             834 non-null    object
 1   nome                                834 non-null    object
 2   cep                                 400 non-null    object
 3   endereco                            430 non-null    object
 4   descricao                           834 non-null    object
 5   funcaoSocial                        834 non-null    object
 6   metaGlobal                          834 non-null    object
 7   dataInicialPrevista                 832 non-null    object
 8   dataFinalPrevista                   832 non-null    object
 9   dataInicialEfetiva                  23 non-null     object
 10  dataFinalEfetiva                    7 non-null      object
 11  dataCadastro                        834 non-null    object

In [350]:
pd.set_option("display.max_columns", None)
df.head()

,idUnico,nome,cep,endereco,descricao,funcaoSocial,metaGlobal,dataInicialPrevista,dataFinalPrevista,dataInicialEfetiva,dataFinalEfetiva,dataCadastro,especie,natureza,naturezaOutras,situacao,descPlanoNacionalPoliticaVinculado,uf,qdtEmpregosGerados,descPopulacaoBeneficiada,populacaoBeneficiada,observacoesPertinentes,isModeladaPorBim,dataSituacao,tomadores,executores,repassadores,eixos,tipos,subTipos,fontesDeRecurso
0,1828.53-16,SISEG - Sistema integrado de segurança,1,,implantação dos sistemas de monitoramento por ...,implantação dos sistemas de monitoramento por ...,Modernização do sistema de segurança do Banco ...,2019-02-04,2024-02-06,None,None,2021-05-12,Recuperação,Projeto,,Cadastrada,None,DF,None,None,None,None,None,2021-05-12,[],"[{'nome': 'BANCO CENTRAL DO BRASIL', 'codigo':...",[],"[{'id': 1, 'descricao': 'Administrativo'}]","[{'id': 5, 'descricao': 'Administrativo', 'idE...","[{'id': 59, 'descricao': 'Obras em Imóveis de ...","[{'origem': 'Federal', 'valorInvestimentoPrevi..."
1,1711.53-18,Reforma do espelho d'água do Edifício-Sede do ...,1,,Reforma do espelho d'água do Edifício-Sede do ...,Reestabelecer a impermeabilização do espelho d...,Recuperação do sistema de impermeabilização e ...,2021-01-04,2021-05-18,None,None,2021-05-04,Reforma,Obra,,Cadastrada,None,DF,None,None,None,None,None,2021-05-04,"[{'nome': 'BANCO CENTRAL DO BRASIL', 'codigo':...","[{'nome': 'BANCO CENTRAL DO BRASIL', 'codigo':...",[],"[{'id': 1, 'descricao': 'Administrativo'}]","[{'id': 5, 'descricao': 'Administrativo', 'idE...","[{'id': 59, 'descricao': 'Obras em Imóveis de ...","[{'origem': 'Federal', 'valorInvestimentoPrevi..."
2,1989.53-01,Elaboração de projetos arquitetônicos e de eng...,1,"Campus Universitário Darcy Ribeiro, Brasília - DF",Contratação de empresa especializada para elab...,Obras para unidades acadêmicas de ensino e pes...,Elaboração de projetos de engenharia e Arquite...,2021-05-18,2021-09-29,None,None,2021-05-17,Construção,Projeto,None,Cadastrada,None,DF,None,None,None,None,None,2021-05-17,[],"[{'nome': 'FUNDACAO UNIVERSIDADE DE BRASILIA',...","[{'nome': 'FUNDACAO UNIVERSIDADE DE BRASILIA',...","[{'id': 1, 'descricao': 'Administrativo'}]","[{'id': 8, 'descricao': 'Educação', 'idEixo': 1}]","[{'id': 46, 'descricao': 'Instituições Federai...","[{'origem': 'Federal', 'valorInvestimentoPrevi..."
3,2004.53-58,SISEG - Sistema integrado de segurança,1,,implantação dos sistemas de monitoramento por ...,implantação dos sistemas de monitoramento por ...,Modernização do sistema de segurança do Banco ...,2019-02-04,2024-02-06,None,None,2021-05-18,Recuperação,Projeto,,Cadastrada,None,DF,None,None,None,None,None,2021-05-18,[{'nome': 'BANCO CENTRAL DO BRASIL-ORC.FISCAL/...,"[{'nome': 'BANCO CENTRAL DO BRASIL', 'codigo':...",[],"[{'id': 1, 'descricao': 'Administrativo'}]","[{'id': 5, 'descricao': 'Administrativo', 'idE...","[{'id': 59, 'descricao': 'Obras em Imóveis de ...","[{'origem': 'Federal', 'valorInvestimentoPrevi..."
4,2617.53-02,CENTRO DE FORMAÇÃO TECNOLOGICA - CFT - CAMPUS ...,1,"Área Especial n° 01, Quadra 16, Cidade do Auto...","construção um galpão com pé direito alto, que ...",atender a comunidade acadêmica do Campus Estru...,aumentar a oferta de cursos técnicos/tecnológi...,2020-12-01,2021-10-27,None,None,2021-06-18,Construção,Obra,,Cadastrada,triplicar as matrículas da educação profission...,DF,None,None,None,None,None,2021-06-18,[{'nome': 'INSTITUTO FED. ED. CIENCIA E TEC. D...,[{'nome': 'INSTITUTO FED. ED. CIENCIA E TEC. D...,"[{'nome': 'MINISTÉRIO DA EDUCAÇÃO', 'codigo': ...","[{'id': 4, 'descricao': 'Social'}]","[{'id': 46, 'descricao': 'Educação', 'idEixo':...","[{'id': 84, 'descricao': 'Educação', 'idTipo':...","[{'origem': 'Federal', 'valorInvestimentoPrevi..."


In [351]:
# use this on each column to check for variable type
df["isModeladaPorBim"].value_counts(dropna=False)

isModeladaPorBim
False    560
None     243
True      31
Name: count, dtype: int64

In [352]:
# categorical variables
df["natureza"] = df["natureza"].astype("category")
df["situacao"] = df["situacao"].astype("category")
df["especie"] = df["especie"].astype("category")
df["uf"] = df["uf"].astype("category")

In [353]:
# date variables
# manually checked fields for formatting errors but found no issues
df["dataInicialPrevista"] = pd.to_datetime(df["dataInicialPrevista"], errors="coerce")
df["dataFinalPrevista"] = pd.to_datetime(df["dataFinalPrevista"], errors="coerce")
df["dataInicialEfetiva"] = pd.to_datetime(df["dataInicialEfetiva"], errors="coerce")
df["dataFinalEfetiva"] = pd.to_datetime(df["dataFinalEfetiva"], errors="coerce")
df["dataCadastro"] = pd.to_datetime(df["dataCadastro"], errors="coerce")
df["dataSituacao"] = pd.to_datetime(df["dataSituacao"], errors="coerce")

In [354]:
# some records have strings instead of ints. these records also have other issues; see below.
# this coerces strings to null
# convert to float instead of int to allow for null values
df["qdtEmpregosGerados"] = pd.to_numeric(df["qdtEmpregosGerados"], errors="coerce")
df["populacaoBeneficiada"] = pd.to_numeric(df["populacaoBeneficiada"], errors="coerce")

In [355]:
df["idLabLivre"] = df.index

### Dealing with nested cols


In [ ]:
# i never quite know how to do this with json. thankfully these are simple flact structures.

nested_cols = [
    "tomadores",
    "executores",
    "repassadores",
    "eixos",
    "tipos",
    "subTipos",
    "fontesDeRecurso",
]


def normalize_nested_column(
    df: pd.DataFrame, id_col: str, nested_col: str
) -> pd.DataFrame:
    def convert_dict_values_to_str(x):
        # pd.json_normalize(df['tomadores'].explode()) converts the id to float, which might cause issues due to floating point imprecision even if converted back to int/str. because of this, i need to convert the values to strings first, which requires some unreadable code:

        if not isinstance(x, list):
            return x
        return [{key: str(value) for key, value in item.items()} for item in x]

    df[nested_col] = df[nested_col].apply(convert_dict_values_to_str)
    exploded = df[[id_col, nested_col]].explode(nested_col)
    exploded = exploded.dropna(subset=[nested_col])
    normalized = pd.json_normalize(exploded[nested_col])
    normalized[id_col] = exploded[id_col].values
    return normalized


def create_entity_table_from_junction_table(
    old_df: pd.DataFrame, prev_index: str, actual_index: str
) -> pd.DataFrame:
    new_df = old_df.dropna(subset=[prev_index])
    new_df.drop_duplicates(subset=[actual_index], inplace=True)
    new_df.set_index(actual_index, inplace=True)
    new_df.drop(prev_index, axis=1, inplace=True)
    return new_df

In [389]:
projeto_tomadores_df = normalize_nested_column(df, "idLabLivre", "tomadores")
tomadores_df = create_entity_table_from_junction_table(
    projeto_tomadores_df, "idLabLivre", "codigo"
)
tomadores_df.head()

,nome
codigo,
25280,BANCO CENTRAL DO BRASIL
25201,BANCO CENTRAL DO BRASIL-ORC.FISCAL/SEG.SOCIAL
26356,INSTITUTO FED. ED. CIENCIA E TEC. DE BRASILIA
2852,MINISTÉRIO DE MINAS E ENERGIA
30108,DEPARTAMENTO DE POLICIA FEDERAL


In [390]:
projeto_executores_df = normalize_nested_column(df, "idLabLivre", "executores")
executores_df = create_entity_table_from_junction_table(
    projeto_executores_df, "idLabLivre", "codigo"
)
executores_df.head()

,nome
codigo,
25280,BANCO CENTRAL DO BRASIL
26271,FUNDACAO UNIVERSIDADE DE BRASILIA
26356,INSTITUTO FED. ED. CIENCIA E TEC. DE BRASILIA
54844,DEPARTAMENTO NACIONAL DE INFRAESTRUTURA DE TRA...
2852,MINISTÉRIO DE MINAS E ENERGIA


In [393]:
projeto_repassadores_df = normalize_nested_column(df, "idLabLivre", "repassadores")
repassadores_df = create_entity_table_from_junction_table(
    projeto_repassadores_df, "idLabLivre", "codigo"
)
repassadores_df.head()

,nome
codigo,
26271,FUNDACAO UNIVERSIDADE DE BRASILIA
244,MINISTÉRIO DA EDUCAÇÃO
2852,MINISTÉRIO DE MINAS E ENERGIA
30108,DEPARTAMENTO DE POLICIA FEDERAL
30911,FUNDO NACIONAL DE SEGURANCA PUBLICA


In [394]:
entidades_df = pd.concat([tomadores_df, executores_df, repassadores_df])
entidades_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 186 entries, 25280 to 36212
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   nome    186 non-null    object
dtypes: object(1)
memory usage: 2.9+ KB


In [ ]:
projeto_fontesDeRecurso_df = normalize_nested_column(
    df, "idLabLivre", "fontesDeRecurso"
)
# nesse caso estava certo o float
projeto_fontesDeRecurso_df["valorInvestimentoPrevisto"] = pd.to_numeric(
    projeto_fontesDeRecurso_df["valorInvestimentoPrevisto"], errors="coerce"
)

fontesDeRecurso_df = create_entity_table_from_junction_table(
    projeto_fontesDeRecurso_df, "idLabLivre", "id"
)
fontesDeRecurso_df

In [361]:
# Check number of unique values for object columns
for col in df.select_dtypes(include=["object"]).columns:
    n_unique = df[col].nunique()
    print(f"{col}: {n_unique} unique values")

idUnico: 712 unique values
nome: 666 unique values
cep: 97 unique values
endereco: 245 unique values
descricao: 651 unique values
funcaoSocial: 519 unique values
metaGlobal: 484 unique values
naturezaOutras: 38 unique values
descPlanoNacionalPoliticaVinculado: 78 unique values
descPopulacaoBeneficiada: 42 unique values
observacoesPertinentes: 4 unique values
isModeladaPorBim: 2 unique values


TypeError: unhashable type: 'list'

In [ ]:
# Check for duplicates based on 'idUnico'
n_duplicates = df["idUnico"].duplicated().sum()
print(f"Number of duplicates by idUnico: {n_duplicates}")

if n_duplicates > 0:
    # Show examples of duplicate records
    duplicate_ids = df[df["idUnico"].duplicated(keep=False)]["idUnico"].unique()
    print("\nExample duplicate records:")
    for id_unico in duplicate_ids[:3]:  # Show up to 3 examples
        print(f"\nRecords with idUnico '{id_unico}':")
        print(
            df[df["idUnico"] == id_unico][
                ["idUnico", "nome", "dataCadastro", "situacao"]
            ].to_string()
        )

Number of duplicates by idUnico: 122

Example duplicate records:

Records with idUnico '3938.53-53':
        idUnico                                                                         nome dataCadastro    situacao
95   3938.53-53  Remanescentes da obra de acessibilidade de demais adequações do Campus Gama   2021-09-23  Cadastrada
306  3938.53-53  Remanescentes da obra de acessibilidade de demais adequações do Campus Gama   2021-09-23  Cadastrada

Records with idUnico '44063.53-80':
         idUnico                                                         nome dataCadastro    situacao
100  44063.53-80  Regularização Fundiária do Sol Nascente, Trecho II, Etapa 1   2024-10-21  Cadastrada
535  44063.53-80  Regularização Fundiária do Sol Nascente, Trecho II, Etapa 1   2024-10-21  Cadastrada

Records with idUnico '58004.53-70':
         idUnico                                                                                             nome dataCadastro    situacao
101  58004.53-70  Const

In [ ]:
# todo manually check if these duplicates are exactly the same and if so drop.

## Load DB


In [ ]:
import sqlite3

con = sqlite3.connect("projeto_investimento.db")
df.to_sql("projeto_investimento", con, if_exists="replace")

### Basic info

Columns: ...

-   strings
    -   ...
-   ints
    -   ...
-   categories
    -   ...
-   nested
    -   ...

Entries: 834-X (+X are duplicated)

.info.
nulls: ...


## Descriptives


## Data Quality


## Analysis


In [ ]:
# todo
# check map
# check dates compared to today

I dont know what these data mean so I chose not to drop any null values.


## Insights


-   not sure how this is collected but `descPlanoNacionalPoliticaVinculado` is clearly confusing for whoever is filling this form. some treat as categorical and some as a text field.
-   some entries have systematic issues, with `descPopulacaoBeneficiada` being template text and `qdtEmpregosGerados` as empty string: `df[(df["qdtEmpregosGerados"] == ' ') & (df['descPopulacaoBeneficiada'] == 'Descrição população beneficiada')]`

    -   warrants further investigation; ask a domain expert.

-   best not to normalize the text here. who knows what is going on...

```json
[
    {
        "codigo": 26,
        "nome": "PRESIDÊNCIA DA REPÚBLICA"
    },
    {
        "codigo": 20000,
        "nome": "Presidencia da República"
    }
]
```

subtipos tem 2 ids que nao match. porra
